In [2]:
# Import packages
import importlib
import os
import time
import numpy as np
import gc

In [3]:
# -------------- PART 0a: CHOOSE CIRCUIT AND SET UP FOLDER --------------                            #<--------- SAME AS CODE1 UP TO...

# Choose circuit
circuit = input("Please enter name of the circuit: ")

# Import circuit config file
config = importlib.import_module(circuit)

# Define subfolder name to work in
folder_name = f"MOSA_{circuit}"

# Create folder if not yet exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Jump to folder
os.chdir(folder_name)

# Prompt new folder name
print(f"Current working directory: {os.getcwd()}")                                                   #<--------- ...UP TO HERE

Current working directory: /Users/nt625/Documents/GitHub/MOSA_bashmethod_2025/MOSA_arneg


In [4]:
# -------------- PART 0d: CHOOSE SENSITIVITY FUNCTIONS --------------                                #<--------- SAME AS CODE1 UP TO...

# Print prompt
print("""
Only two sensitivity functions are present:
0. |S_alpha_xss|
1. |S_n_xss|
MOSA will anneal this pair.
""")

# Choose pair of functions
choice1 = 0
choice2 = 1

# List of sensitivity function names
sensitivity_labels = [
    "|S_alpha_xss|",
    "|S_n_xss|"]

# Save function names for later use
label1 = sensitivity_labels[choice1]
label2 = sensitivity_labels[choice2]


# -------------- PART 0e: CHANGING DIRECTORIES --------------


# Define subfolder name to work in
subfolder_name = f"MOSA_sensfuncs_{choice1}_and_{choice2}"

# Create folder if not yet exist
if not os.path.exists(subfolder_name):
    os.makedirs(subfolder_name)

# Jump to folder
os.chdir(subfolder_name)

# Prompt new folder name
print(f"Current working directory: {os.getcwd()}")                                                   #<--------- ...UP TO HERE


Only two sensitivity functions are present:
0. |S_alpha_xss|
1. |S_n_xss|
MOSA will anneal this pair.

Current working directory: /Users/nt625/Documents/GitHub/MOSA_bashmethod_2025/MOSA_arneg/MOSA_sensfuncs_0_and_1


In [4]:
# -------------------- NEW STUFF --------------------

In [5]:
runs = int(input("How many MOSA runs did you run: "))

In [6]:
# Define function that returns a 2D numpy array containing rows from arr2 that are not in arr1, where arr1 and arr2 are 2D NumPy arrays
def rows_not_in_arr1(arr1, arr2):
    dtype = [('f0', arr2.dtype), ('f1', arr2.dtype)]
    arr1_struct = arr1.view(dtype)
    arr2_struct = arr2.view(dtype)
    unique_rows = np.setdiff1d(arr2_struct, arr1_struct).view(arr2.dtype).reshape(-1, arr2.shape[1])
    return unique_rows

In [7]:
# Define original grid points
alpha_min      = 0.01
alpha_max      = 50
alpha_no       = 1000
alpha_vals = np.linspace(alpha_min,alpha_max,alpha_no)
n_min      = 0.01
n_max      = 10
n_no       = 1000
n_vals = np.linspace(n_min,n_max,n_no)
grid_x, grid_y = np.meshgrid(alpha_vals,n_vals)
grid_original = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

In [8]:
# Empty array to keep track of what points in search space we've already accumulated as we progress through runs
accumulating_searchspace = np.empty((0, 2))

# For each run in our total number of runs
for run in range(1, runs + 1):

    # Load the parameter values for that run's Pareto front
    pareto_alpha = np.load(f"pareto_alpha_run{run}.npy", allow_pickle=True)
    pareto_n = np.load(f"pareto_n_run{run}.npy", allow_pickle=True)
    
    # Create 2D points from the loaded data
    points = np.array(list(zip(pareto_alpha, pareto_n)))
    
    # Compute the bounding box from the points
    min_x, min_y = np.min(points, axis=0)
    max_x, max_y = np.max(points, axis=0)
    
    # Obtain mask
    mask = (
        (grid_original[:, 0] >= min_x) & (grid_original[:, 0] <= max_x) &
        (grid_original[:, 1] >= min_y) & (grid_original[:, 1] <= max_y)
    )
    
    # Keep points inside mask
    grid_masked = grid_original[mask]
    
    #  Now we have
    #                   --------------------------------
    #                  |   alpha values  |   n values   |
    #                  |         #       |       #      |
    #  grid_masked  =  |         #       |       #      |
    #                  |         #       |       #      |
    #                  |         #       |       #      |
    #                   --------------------------------

    # Keep new points only
    grid_new = rows_not_in_arr1(accumulating_searchspace, grid_masked)
    
    # Save data
    np.save(f"grid_new_run{run}.npy", grid_new)
    
    # Accumulate new points
    accumulating_searchspace = np.vstack([accumulating_searchspace, grid_new])
    
    # Print shape
    print(f"Shape after Run {run}: {accumulating_searchspace.shape}")

Shape after Run 1: (42, 2)
Shape after Run 2: (206586, 2)
Shape after Run 3: (336006, 2)
Shape after Run 4: (336045, 2)
Shape after Run 5: (336591, 2)


In [ ]:
# Convert final array into a set of valid (alpha, n) pairs for fast lookup
valid_points = set(map(tuple, accumulating_searchspace))

# Create a boolean mask (True if the point is in the final valid array)
mask = np.array([tuple(point) in valid_points for point in grid_original])

# Create a new grid where invalid points are replaced with NaN
grid_final = np.copy(grid_original)
grid_final[~mask] = np.nan  # Set invalid points to NaN

# Reshape back to 2D format if needed (optional)
grid_final_x = grid_final[:, 0].reshape(grid_x.shape)
grid_final_y = grid_final[:, 1].reshape(grid_y.shape)

# Save masked grid
np.save("grid_final.npy", grid_final)